### Importar librerias

In [ ]:
!pip install fbprophet

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from datetime import datetime, timedelta
from pandas import DataFrame
import seaborn as sns
%matplotlib inline
from fbprophet import Prophet
import warnings
warnings.filterwarnings("ignore")

### Cargar Datos 

In [ ]:
url = 'https://taylorinteractivo.com.co/proyecto-supermercado/data-v3.csv'
dataset = pd.read_csv(url, delimiter=',', encoding= 'latin-1')
# Transforma en formato fecha a la base
dataset['fecha'] = dataset['fecha'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
# Se carga xlsx con los datos de los dias festivos en colombia desde 2020 hasta el 2023
Festivos= pd.read_excel('/content/Festivos.xlsx')

### Limpieza y preparacion de los datos 

In [ ]:
# Eliminamos las filas que tengan una cantidad superior a 100
dataset = dataset.drop(dataset[dataset['cantidad']>100].index)
# Eliminamos productos genericos
dataset = dataset.drop(dataset[dataset['descrip']=='PRODUCTOS VARIOS'].index)
dataset = dataset.drop(dataset[dataset['descrip']=='PRODUCTOS DE 1200'].index)
#dataset.groupby(['descrip'])['cantidad'].agg('sum')

In [ ]:
# añadimos columnas de mes y año
dataset['Mes']=dataset['fecha'].dt.month
dataset['Año']=dataset['fecha'].dt.year

In [ ]:
# agrupamos por producto , fecha y agrupamos por cantidad
df_fecha=dataset.groupby(['descrip','Año','Mes'])['cantidad'].agg('sum')
df_fecha = df_fecha.to_frame()
df_fecha = df_fecha.reset_index()

df_fecha=df_fecha.groupby(['descrip'])['Mes'].agg('count')
df_fecha = df_fecha.to_frame()
df_fecha = df_fecha.reset_index()

# Eliminamos los productos con menos de 6 meses de datos
elim =df_fecha[df_fecha['Mes'] < 6]
elim=elim['descrip'].unique()

for j in elim:
  dataset = dataset.drop(dataset[dataset['descrip']==j].index)

In [ ]:
# agrupamos por producto , fecha y agrupamos por cantidad
df_estructura=dataset.groupby(['descrip','fecha'])['cantidad'].agg('sum')
df_estructura = df_estructura.to_frame()
df_estructura = df_estructura.reset_index()

In [ ]:
# filtramos los productos 
productos=df_estructura['descrip'].unique()
#data_df = pd.DataFrame(productos)
#df_estructura.to_excel('test2.xlsx',index=False)

In [ ]:
len(productos)

2576

## Modelo Fbprofhet

In [ ]:
# Funcion que recibe el dataframe a analizar los periodos a predicir y base de festivo
# Retorna los datos de la prediccion
def Prediccion_ventas(data, periodos,festivos):  
    model = Prophet(holidays=festivos , yearly_seasonality=True)
    model.fit(data)
    future   = model.make_future_dataframe(periods = periodos)
    forecast = model.predict(future)
    #figure   = model.plot(forecast, xlabel = "Fecha", ylabel = "Ventas")
    #figure2  = model.plot_components(forecast)
    # Filtramos los datos predichos
    Prediccion = forecast[['ds','yhat']].rename(columns = {'yhat': 'y'})
    Prediccion=Prediccion.iloc[len(Prediccion)-periodos:]
    #Prediccion['y']= Prediccion['y'].apply(np.floor)
    Prediccion['y']=Prediccion['y'].where(Prediccion['y'] > 0 , 0 )
   

    return Prediccion

In [ ]:
# Funcion que recibe el producto a analizar y lod evuelve con la estructura filtrada

def Estructura(datos,produc):
  data=datos[datos["descrip"] == produc]
  data=data.groupby(['fecha'])['cantidad'].agg('sum')
  # Se completa con la fechas faltantes dentro del rango de toma de la muestra
  fin=len(data)-1
  # se crea el rango de fechas
  idx = pd.date_range(data.index[0], data.index[fin])
  data.index = pd.DatetimeIndex(data.index)
  data = data.reindex(idx, fill_value=0)
  data=pd.DataFrame(data)
  data['Fecha'] = data.index
  data = data.reset_index(drop= True)
  data = data[['Fecha', 'cantidad']].rename(columns = {'Fecha': 'ds', 'cantidad': 'y'})
  data = data.sort_values('ds')
  return data


In [ ]:
def Agrupar(data,Prediccion,producto):
  inicio=data['ds'].iloc[0]
  fin=data['ds'].iloc[len(data)-1]
  fin_p=Prediccion['ds'].iloc[len(Prediccion)-1]
  meses=((fin.year-inicio.year)*12+fin.month -inicio.month)

  final=data.append(Prediccion)
  final=final.groupby(final['ds'].dt.strftime('%Y%m'))['y'].sum()
  final=pd.DataFrame(final)
  final['Fecha'] = final.index
  final = final.reset_index(drop= True)
  final['Fecha'] = final['Fecha'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m'))
  final = final[['Fecha','y']].rename(columns = {'Fecha': 'Fecha_mes' ,'y': 'Ventas'})
  final.insert(0, 'Producto', producto)
  final["F_Modelo"]=0
  final["F_Modelo"].iloc[0]=inicio
  final["F_Modelo"].iloc[meses]=fin
  final["F_Modelo"].iloc[len(final)-1]=fin_p

  for e in range(meses,len(final)):
    final['Ventas'].iloc[e]=round(final['Ventas'].iloc[e])


  return final

### Ejecutar predicciones

In [ ]:
# primer elemento 
periodos=90
producto_eval=productos[0]

data=Estructura(df_estructura,producto_eval)
Prediccion=Prediccion_ventas(data, periodos,Festivos)
Todos=Agrupar(data,Prediccion,producto_eval)

# separar datos base y predichos
numelem=len(Todos)
corte=0
for x in range(1,numelem-1):
  temp=Todos['F_Modelo'].iloc[x]
  if temp != 0:
    corte=x+1
corte=numelem-corte  

Reales= Todos.iloc[:len(Todos)-corte]
Predichos= Todos.iloc[len(Todos)-corte:]

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
periodos=90

for e in range(1,len(productos)):
  var=int(e)
  producto_eval=productos[var]
  print(type(producto_eval))
  data=Estructura(df_estructura,producto_eval)
  Prediccion=Prediccion_ventas(data, periodos,Festivos)
  temporal=Agrupar(data,Prediccion,producto_eval)
  # separar datos base y predichos
  numelem=len(temporal)
  corte=0
  for x in range(1,numelem-1):
    temp=temporal['F_Modelo'].iloc[x]
    if temp != 0:
      corte=x+1
  corte=numelem-corte 
  RealesT= temporal.iloc[:len(temporal)-corte] 
  PredichosT= temporal.iloc[len(temporal)-corte:]
  Reales=Reales.append(RealesT)
  Predichos=Predichos.append(PredichosT)


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<class 'str'>


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<class 'str'>


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

<class 'str'>


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<class 'str'>


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<class 'str'>


In [ ]:
Reales['Año']=Reales['Fecha_mes'].dt.year 
Reales['Mes']=Reales['Fecha_mes'].dt.month 

In [ ]:
Predichos['Año']=Predichos['Fecha_mes'].dt.year 
Predichos['Mes']=Predichos['Fecha_mes'].dt.month 

In [ ]:
Reales.to_excel('Reales.xlsx',index=False)
Predichos.to_excel('Predichos.xlsx',index=False)
#productos.to_excel('Productos.xlsx',index=False)

In [ ]:
productos1=pd.DataFrame(productos)

In [ ]:
productos1.to_excel('Productos.xlsx',index=False)